# Solubility Prediction by Random Forest Algorithem Using RDKit 2D and 3D Descriptors

**The formula is modeling solubility as a linear function of three factors:**

- logP (lipophilicity): Solubility decreases as logP increases (more hydrophobic molecules are less soluble in water).

- Molecular weight (mol_wt): Solubility increases slightly as molecular weight increases.

- Hydrogen bond donors (h_bond_donors): Solubility decreases as the number of hydrogen bond donors increases.

## 1. Create database

In [1]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Descriptors
import numpy as np

# Generate a list of SMILES (valid and a few invalid)
smiles_list = [
    "CCO", "O=C(O)c1ccccc1OC", "CN1C=NC2=C1C(=O)N(C(=O)N2C)C", 
    "C([C@@H]([C@@H]([C@H](O)CO)O)O)O", "CC(=O)OC1=CC=CC=C1C(=O)O",
    "CC(C)CC1=CC=C(C=C1)O", "C1=CC=C(C=C1)O", "C1CCCCC1", 
    "C1=CC=NC(=C1)N", "C(CO)NC(C)C", "InvalidSMILES123", 
    "C1CCOC1", "C1=CN=C2C(=N1)C=NC=N2", "NaCl", 
    # ... Add more SMILES (repeat with variations) ...
]

# Generate synthetic solubility based on descriptors
data = []
for smiles in smiles_list * 10:  # Repeat to create 200+ entries
    mol = Chem.MolFromSmiles(smiles)
    if mol is not None:
        # Calculate descriptors
        logp = Descriptors.MolLogP(mol)
        mol_wt = Descriptors.MolWt(mol)
        h_bond_donors = Descriptors.NumHDonors(mol)
        
        # Simulate solubility: higher solubility for lower logP, lower molecular weight
        solubility = 0.5 - 0.02 * logp + 0.001 * mol_wt - 0.1 * h_bond_donors
        solubility += np.random.normal(0, 0.05)  # Add noise
        
        # Normalize: Clip solubility between 0 and 1
        solubility = np.clip(solubility, 0.05, 0.95)
        data.append({"SMILES": smiles, "Solubility": round(solubility, 2)})

# Create DataFrame and save
df = pd.DataFrame(data)
df.to_csv("sample_solubility_2.csv", index=False)

[17:41:33] SMILES Parse Error: syntax error while parsing: InvalidSMILES123
[17:41:33] SMILES Parse Error: check for mistakes around position 3:
[17:41:33] InvalidSMILES123
[17:41:33] ~~^
[17:41:33] SMILES Parse Error: Failed parsing SMILES 'InvalidSMILES123' for input: 'InvalidSMILES123'
[17:41:33] SMILES Parse Error: syntax error while parsing: NaCl
[17:41:33] SMILES Parse Error: check for mistakes around position 2:
[17:41:33] NaCl
[17:41:33] ~^
[17:41:33] SMILES Parse Error: Failed parsing SMILES 'NaCl' for input: 'NaCl'
[17:41:33] SMILES Parse Error: syntax error while parsing: InvalidSMILES123
[17:41:33] SMILES Parse Error: check for mistakes around position 3:
[17:41:33] InvalidSMILES123
[17:41:33] ~~^
[17:41:33] SMILES Parse Error: Failed parsing SMILES 'InvalidSMILES123' for input: 'InvalidSMILES123'
[17:41:33] SMILES Parse Error: syntax error while parsing: NaCl
[17:41:33] SMILES Parse Error: check for mistakes around position 2:
[17:41:33] NaCl
[17:41:33] ~^
[17:41:33] SMILE

## 2. Prepare the data and calculate descriptions
The amount of features we import in our file has great impact. 
Too much features without PCA leads to overfitting. This is what I saw when I imported 3d data in current code.  
  
  Warning: I didn't normalize new features. I should check them again.  
  So I have 221 features with deifferent weights!

### Do we have outliers?
If the output of the following cell shows many outliers, Robust Scaling is the best choice for normalizing. Otherwise, use Standard Scaling.

In [2]:
# Load the dataset
data = pd.read_csv("molecular_descriptors.csv")

# Exclude non-numeric columns (e.g., SMILES, Solubility)
numeric_data = data.select_dtypes(include=[np.number])

# Detect outliers using IQR
Q1 = numeric_data.quantile(0.25)
Q3 = numeric_data.quantile(0.75)
IQR = Q3 - Q1

# Identify outliers
outliers = ((numeric_data < (Q1 - 1.5 * IQR)) | (numeric_data > (Q3 + 1.5 * IQR))).sum()

# Print the number of outliers per column
print(f"the number of outliers per column: {outliers[outliers > 0]}")

the number of outliers per column: MinAbsEStateIndex         10
SPS                       30
MinPartialCharge          10
MaxAbsPartialCharge       10
FpDensityMorgan1          10
                          ..
fr_para_hydroxylation     30
fr_phenol                 20
fr_phenol_noOrthoHbond    20
fr_pyridine               10
Solubility                 9
Length: 81, dtype: int64


In [3]:
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import AllChem

# Load the dataset
data = pd.read_csv("sample_solubility_2.csv")

def get_descriptors(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol:
        desc = Descriptors.CalcMolDescriptors(mol)
        desc["PolarSurfaceArea"] = rdMolDescriptors.CalcTPSA(mol)  # 2D Descriptor

        # Generate 3D Conformer (required for 3D descriptors)
        mol = Chem.AddHs(mol)  # Add hydrogens
        if AllChem.EmbedMolecule(mol, AllChem.ETKDG()) == 0:  # Ensure embedding succeeds
            desc["RadiusOfGyration"] = rdMolDescriptors.CalcRadiusOfGyration(mol)  # 3D Descriptor
            desc["Asphericity"] = rdMolDescriptors.CalcAsphericity(mol)  # 3D Descriptor
            desc["Eccentricity"] = rdMolDescriptors.CalcEccentricity(mol)  # 3D Descriptor
        else:
            desc["RadiusOfGyration"] = None
            desc["Asphericity"] = None
            desc["Eccentricity"] = None

        return desc
    return None

# Apply the function to create a descriptor DataFrame
descriptor_list = []
for idx, row in data.iterrows():
    desc = get_descriptors(row["SMILES"])
    if desc is not None:
        desc["SMILES"] = row["SMILES"]  # Track valid SMILES
        desc["Solubility"] = row["Solubility"]
        descriptor_list.append(desc)

# Create final DataFrame
df = pd.DataFrame(descriptor_list).dropna()
print(f"Valid molecules: {len(df)}")

# Save to CSV
df.to_csv("molecular_descriptors.csv", index=False)
print("Saved descriptors to molecular_descriptors.csv")


Valid molecules: 120
Saved descriptors to molecular_descriptors.csv


## 3. Preprocess    
Main parts:  
1. Split
2. Normalize (if didn't in previous part)  
Had major problams with this part.  


The changes I can make to improve my results:    
- remove ouliers: Didn't work well with my sample.  
- Selecting best features while omitting others to avoid overfitting. I have to recieve the name of selected features.  
- Handle missing values.

In [4]:
# Load your generated descriptor data
df = pd.read_csv("molecular_descriptors.csv")

# Split into features (X) and target (y)
X = df.drop(["SMILES", "Solubility"], axis=1)
y = df["Solubility"]

# Handle missing values (if any)
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
X = imputer.fit_transform(X)

# Split into train/test sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
print(f"X_train = {X_train.shape} , X_test = {X_test.shape}, y_train = {y_train.shape}, y_test = {y_test.shape}")

X_train = (96, 221) , X_test = (24, 221), y_train = (96,), y_test = (24,)


In [6]:
# Normalize the data with RobustScaler as we have outliers.

from sklearn.preprocessing import RobustScaler

# Initialize the scaler
scaler = RobustScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data (if applicable)
X_test_scaled = scaler.transform(X_test)

# Check the scaling
print(f"X_train_scaled = {X_train_scaled.shape} , X_test_scaled = {X_test_scaled.shape}")

X_train_scaled = (96, 221) , X_test_scaled = (24, 221)


### feature selection

In [7]:
from sklearn.decomposition import PCA

#Decreasing the number of features from 221 to the best number of components:
pca = PCA(n_components=17)
pca.fit(X_train_scaled)

X_new_train= pca.transform(X_train_scaled)
X_new_test = pca.transform(X_test_scaled)
#y_new_train = y_train_clean
X_train.shape , X_new_train.shape, #y_new_train.shape
#Check the decrease in features

((96, 221), (96, 17))

## 4. Training the model
I used GridSearchCV to optimize the parameters of my randomforest model.

In [8]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Define parameter grid:
# If it took so long to run, you can reduce the number of parameters.
param_grid = {
    'n_estimators': [200, 300],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 0.5]
}

# Initialize and fit grid search
model = RandomForestRegressor(random_state=0)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='neg_root_mean_squared_error', error_score='raise')
grid_search.fit(X_new_train, y_train)

# Get best model
best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)

# Now use best_model for predictions
best_model.fit(X_new_train, y_train)

Best Parameters: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}


RandomForestRegressor(max_depth=10, max_features=0.5, min_samples_split=10,
                      n_estimators=200, random_state=0)

### In Case of running into bugs, Check these cells. Oherwise skip them.

In [9]:
#Verify Data Types
print(y_train.dtype)  # Should output `float64` or `int64`
print(y_test.dtype)   # Should not output `object`

float64
float64


In [10]:
print(type(X_train))  # If it's a DataFrame, use .values. If it's an array, skip.

# .values:
#Ensure Input Shapes
# Check that X_train, X_test, y_train, and y_test are NumPy arrays or Pandas DataFrames (not lists or other objects):
# Convert to NumPy arrays if needed
#X_train = X_train.values
#X_test = X_test.values
#y_train = y_train.values.ravel()  # Flatten to 1D array
#y_test = y_test.values.ravel()

<class 'numpy.ndarray'>


## 5. Evaluate

- RMSE (Root Mean Squared Error): Measures the average prediction error (lower = better).  
Example: RMSE = 0.5 means predictions are off by ~0.5 units (e.g., logS) on average.<br>
<br> 

- R² (R-squared): Measures how much variance your model explains (1 = perfect, 0 = no better than the mean). <br>Train R² ≈ Test R² (e.g., Train R² = 0.85, Test R² = 0.80). Test R² ≥ 0.7 is often good.

In [11]:
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# Let's define a function that works for testing each regression model.
def calculate_regression_metrics(y_train, y_test, y_pred_train, y_pred_test):
    # Calculate metrics for training set
    mse_train = mean_squared_error(y_train, y_pred_train)
    rmse_train = np.sqrt(mse_train)  # Compute RMSE manually
    #rmse_train = mean_squared_error(y_train, y_pred_train, squared=False)
    r2_train = r2_score(y_train, y_pred_train)
    
    # Calculate metrics for test set
    mse_test = mean_squared_error(y_train, y_pred_train)
    rmse_test = np.sqrt(mse_test)  # Compute RMSE manually
    #rmse_test = mean_squared_error(y_test, y_pred_test, squared=False)
    r2_test = r2_score(y_test, y_pred_test)
    
    print(f"Train RMSE: {rmse_train:.2f}, Train R²: {r2_train:.2f}")
    print(f"Test RMSE: {rmse_test:.2f}, Test R²: {r2_test:.2f}")

In [12]:
# Predict solubilities
y_pred_train_rf = best_model.predict(X_new_train)
y_pred_test_rf = best_model.predict(X_new_test)

# Evaluate
print("Best Parameters:", grid_search.best_params_)

calculate_regression_metrics(y_train, y_test, y_pred_train_rf, y_pred_test_rf)

Best Parameters: {'max_depth': 10, 'max_features': 0.5, 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Train RMSE: 0.04, Train R²: 0.91
Test RMSE: 0.04, Test R²: 0.71


### You can change the number of PCA components(features) if the model is overfitting.
1. Use the number you recieve as number of PCA components in the following cell in the pca = PCA(n_components=17)
2. Run the model and Evaluate again

In [13]:
# Finding the best number of PCA components(features).
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score

best_score = -1
for n in range(5, 21):  # Test PCA components from 5 to 20
    pca = PCA(n_components=n)
    X_train_pca = pca.fit_transform(X_train_scaled)
    scores = cross_val_score(model, X_train_pca, y_train, cv=5, scoring='r2')
    if scores.mean() > best_score:
        best_score = scores.mean()
        best_n = n

print(f"Best number of PCA components: {best_n}")

Best number of PCA components: 20
